# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `pyclanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g. `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g. `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for sequence containers (i.e. `pair`, `array`, `vector`, `deque`, `forward_list` and `list` of the `std` namespace) and associative containers (`set`, `multiset`, `map`, `multimap`, `unordered_set`, `unordered_multiset`, `unordered_map` and `unordered_multimap` of the `std` namespace).
These template instantiations are done for *C++* fundamental types (`bool`, `signed char`, `unsigned char`, `char`, `wchar_t`, `int` (with sign modifiers `signed` and `signed` combined or not with size modifiers `short`, `long` and `long long`), `float`, `double`, `long double`) and strings (`string`, `wstring` of the `std` namespace).
For ordered associative containers both `std::less` and `std::greater` comparators are used.
We here only illustrate the procedure on the `std::vector` template class.
For the complete procedure refers to the `AutoWIG.py` file situed at the root of the **PySTL** [repository](https://github.com/StatisKit/PySTL).

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [1]:
!conda build -q STL/conda/libstatiskit_stl -c statiskit
!conda install -y -q libstatiskit_stl --use-local -c statiskit

BUILD START: libstatiskit_stl-1.0.0-0

The following NEW packages will be INSTALLED:

    libdev:       1.0.0-py27_0  local
    openssl:      1.0.2k-1           
    path.py:      10.3.1-py27_0      
    pip:          9.0.1-py27_1       
    python:       2.7.13-0           
    python-scons: 2.5.0-py27_0  local
    pyyaml:       3.12-py27_0        
    readline:     6.2-2              
    setuptools:   27.2.0-py27_0      
    sqlite:       3.13.0-0           
    tk:           8.5.18-0           
    wheel:        0.29.0-py27_0      
    yaml:         0.1.6-0            
    zlib:         1.2.8-3            


latest version is 2.1.12. Run

conda update -n root conda-build

to get the latest version.

Package: libstatiskit_stl-1.0.0-0
source tree in: /home/pfernique/.miniconda/conda-bld/libstatiskit_stl_1494421751287/work
+ source /home/pfernique/.miniconda/bin/activate /home/pfernique/.miniconda/conda-bld/libstatiskit_stl_1494421751287/_b_env_placehold_placehold_placehold_placehold_

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations.

In [2]:
!pygmentize STL/src/cpp/STL.h

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef unsigned long int Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generat

Once these preliminaries done, we can proceed to the actual generation of wrappers for the **PySTL** library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

We need then to install the *C++* headers. This is done using the `cpp` target in **SCons**.

In [3]:
!scons cpp -C STL

scons: Entering directory `/home/pfernique/Desktop/develop/AutoWIG/doc/examples/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
scons: `cpp' is up to date.
scons: done building targets.


Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [4]:
!scons autowig -c -C STL
!scons autowig -C STL

scons: Entering directory `/home/pfernique/Desktop/develop/AutoWIG/doc/examples/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Cleaning targets ...
Removed /home/pfernique/.miniconda/envs/statiskit-dev/include/statiskit/stl/STL.h
Removed build/src/cpp/STL.os
Removed /home/pfernique/.miniconda/envs/statiskit-dev/lib/libstatiskit_stl.so
Removed /home/pfernique/.miniconda/envs/statiskit-dev/lib/python2.7/site-packages/autowig/site/controller/statiskit_stl.py
Removed /home/pfernique/.miniconda/envs/statiskit-dev/lib/python2.7/site-packages/autowig/site/generator/statiskit_stl.py
Removed /home/pfernique/.miniconda/envs/statiskit-dev/lib/python2.7/site-packages/autowig/site/ASG/statiskit_stl.pkl
scons: done cleaning targets.
scons: Entering directory `/home/pfernique/Desktop/develop/AutoWIG/doc/examples/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/STL.h" as "/home

Here is an example of the generated wrappers.
We here present the wrappers for the `std::vector< int >` class.

In [5]:
import pickle
import os
import autowig

with open(os.path.join(autowig.__path__[0], 'site', 'ASG', 'statiskit_stl.pkl'), 'r') as filehandler:
    asg = pickle.load(filehandler)

In [6]:
!pygmentize STL/src/py/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

#include "_stl.h"



namespace autowig
{

    void method_decorator_bb1e0852f2ca56c094260a03787426c7(class ::std::vector< int, class ::std::allocator< int > > & instance, ::std::vector< int, class ::std::allocator< int > >::size_type  param_in_0, int param_out) { instance.at(param_in_0) = param_out; }
    void method_decorator_7ec1ac72b0b05f3a9707175bcd5da0bd(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.front() = param_out; }
    void method_decorator_ed1cf37568ed54cbbd326e6ccbe5f27d(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.back() = param_out; }
}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::std::vector< int, class ::std::allocator< int > > const volatile * get_pointer<class ::std::vector< int, class ::std::allocator< int > > const volatile >(class ::std::vector< int, class ::std::allocator< int > > const volatile *c) { return c; }
}
 

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [7]:
!conda build STL/conda/python-statiskit_stl -c statiskit 
!conda install -y python-statiskit_stl --use-local -c statiskit --force

BUILD START: python-statiskit_stl-3.3.1-py27_0
    (actual version deferred until further download or env creation)
updating index in: /home/pfernique/.miniconda/conda-bld/linux-64
updating index in: /home/pfernique/.miniconda/conda-bld/noarch

The following NEW packages will be INSTALLED:

    coverage:         4.3.4-py27_0       
    icu:              54.1-0             
    libboost:         1.61.0-py27_0 local
    libdev:           1.0.0-py27_0  local
    librun:           1.0.0-py27_0  local
    libstatiskit_stl: 1.0.0-0       local
    nose:             1.3.7-py27_1       
    openssl:          1.0.2k-1           
    path.py:          10.3.1-py27_0      
    pip:              9.0.1-py27_1       
    python:           2.7.13-0           
    python-dev:       1.0.0-py27_0  local
    python-scons:     2.5.0-py27_0  local
    pyyaml:           3.12-py27_0        
    readline:         6.2-2              
    setuptools:       27.2.0-py27_0      
    sqlite:           3.13.0-0      

g++ -o /home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/lib/libstatiskit_stl.so -shared build/src/cpp/STL.os -L/home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/lib
Install file: "src/py/statiskit/stl/generator.py" as "/home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/lib/python2.7/site-packages/statiskit/stl/generator.py"
Install file: "src/py/statiskit/stl/__init__.py" as "/ho

In file included from /home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/include/boost/python/object/function_handle.hpp:10:0,
                 from /home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/include/boost/python/converter/arg_to_python.hpp:19,
                 from /home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/include/boost/python/call.hpp:15,
                 from /ho

g++ -o build/src/py/wrapper_d48105936d4f5d09a78d2aa9f878fdb8.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/include -I/home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/include/python2.7 build/src/py/wrapper_d48105936d4f5d09a78d2aa9f878fdb8.cpp
g++ -o build/src/py/wrapper_448c20257e485acda59dc59305fceb58.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/pfernique/

g++ -o build/src/py/wrapper_6436891c9b6854f494789a812891cbe5.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/include -I/home/pfernique/.miniconda/conda-bld/python-statiskit_stl_1494421927494/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/include/python2.7 build/src/py/wrapper_6436891c9b6854f494789a812891cbe5.cpp
g++ -o build/src/py/wrapper_c047f2c3135554ceb57f166fd404cfc8.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/pfernique/

Test set initialization ... ok
Test set manipulation ... ok
Test set latex representation ... ok
Test set string representation ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.007s

OK
TEST END: /home/pfernique/.miniconda/conda-bld/linux-64/python-statiskit_stl-3.3.1-py27_0.tar.bz2
INFO conda_build.config:__exit__(432): --dirty flag not specified.  Removing build folder after successful build/test.

# If you want to upload package(s) to anaconda.org later, type:

anaconda upload /home/pfernique/.miniconda/conda-bld/linux-64/python-statiskit_stl-3.3.1-py27_0.tar.bz2

# To have conda build upload to anaconda.org automatically, use
# $ conda config --set anaconda_upload yes

anaconda_upload is not set.  Not uploading wheels: []



####################################################################################
Source and build intermediates have been left in /home/pfernique/.miniconda/conda-bld.
There are currently 302 accumulated.
To re

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [8]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

[-1, 0, 1]

In [9]:
list(v)

[-1, 0, 1]

In [10]:
v[0]

-1

In [11]:
v[0] = -2
v[0]

-2

In [12]:
VectorInt([0, 1])

[0, 1]